In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt

In [11]:
datatrx = pd.read_csv('../input/uas-rfm/UAS_RFM.csv', sep=';', parse_dates=['TrxDate'])
datatrx.head()

,CardID,TrxDate,Amount
0,1000152,2021-02-22,670970
1,1000158,2021-12-06,396433
2,1000156,2021-05-24,114270
3,1000115,2021-05-30,420477
4,1000164,2021-06-23,361661


In [14]:
datatrx.head(10)

,CardID,TrxDate,Amount
0,1000152,2021-02-22,670970
1,1000158,2021-12-06,396433
2,1000156,2021-05-24,114270
3,1000115,2021-05-30,420477
4,1000164,2021-06-23,361661
5,1000136,2021-05-05,246831
6,1000147,2021-12-04,794326
7,1000118,2021-03-18,237591
8,1000167,2021-05-16,169632
9,1000160,2021-01-04,370557


In [16]:
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199 entries, 0 to 198
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 4.8 KB


In [17]:
print(datatrx['TrxDate'].min(), datatrx['TrxDate'].max())

2021-01-02 00:00:00 2021-12-06 00:00:00


In [18]:
sd = dt.datetime(2021,12,10)
datatrx['hist']=sd - datatrx['TrxDate']
datatrx['hist'].astype('timedelta64[D]')
datatrx['hist']=datatrx['hist'] / np.timedelta64(1, 'D')
datatrx.head(10)

,CardID,TrxDate,Amount,hist
0,1000152,2021-02-22,670970,291.0
1,1000158,2021-12-06,396433,4.0
2,1000156,2021-05-24,114270,200.0
3,1000115,2021-05-30,420477,194.0
4,1000164,2021-06-23,361661,170.0
5,1000136,2021-05-05,246831,219.0
6,1000147,2021-12-04,794326,6.0
7,1000118,2021-03-18,237591,267.0
8,1000167,2021-05-16,169632,208.0
9,1000160,2021-01-04,370557,340.0


In [19]:
datatrx=datatrx[datatrx['hist'] < 730]
datatrx.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 199 entries, 0 to 198
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   CardID   199 non-null    int64         
 1   TrxDate  199 non-null    datetime64[ns]
 2   Amount   199 non-null    int64         
 3   hist     199 non-null    float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 7.8 KB


In [20]:
rfmTable = datatrx.groupby('CardID').agg({'hist': lambda x:x.min(), # Recency
                                        'CardID': lambda x: len(x), # Frequency
                                        'Amount': lambda x: x.sum()}) # Monetary Value

rfmTable.rename(columns={'hist': 'recency', 
                         'CardID': 'frequency', 
                         'Amount': 'monetary_value'}, inplace=True)
rfmTable.head(10)

,recency,frequency,monetary_value
CardID,,,
1000101,98.0,1,157999
1000102,206.0,3,1750216
1000103,338.0,1,219348
1000104,159.0,2,1070546
1000105,5.0,5,3697893
1000106,69.0,2,1112120
1000107,192.0,3,1027622
1000108,180.0,3,644940
1000109,96.0,2,1119154


In [22]:
quartiles = rfmTable.quantile(q=[0.15,0.30,0.45])
print(quartiles, type(quartiles))

      recency  frequency  monetary_value
0.15     21.5        1.0        538505.0
0.30     99.0        2.0        827777.0
0.45    163.5        2.0       1049084.0 <class 'pandas.core.frame.DataFrame'>


In [23]:
def RClass(x,p,d):
    if x <= d[p][0.15]:
        return 4
    elif x <= d[p][0.30]:
        return 3
    elif x <= d[p][0.45]: 
        return 2
    else:
        return 1
    
def FMClass(x,p,d):
    if x <= d[p][0.15]:
        return 1
    elif x <= d[p][0.30]:
        return 2
    elif x <= d[p][0.45]: 
        return 3
    else:
        return 4

In [25]:
rfmSeg = rfmTable
rfmSeg['R_Quartile'] = rfmSeg['recency'].apply(RClass, args=('recency',quartiles,))
rfmSeg['F_Quartile'] = rfmSeg['frequency'].apply(FMClass, args=('frequency',quartiles,))
rfmSeg['M_Quartile'] = rfmSeg['monetary_value'].apply(FMClass, args=('monetary_value',quartiles,))

In [26]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,98.0,1,157999,3,1,1,311
1000102,206.0,3,1750216,1,4,4,144
1000103,338.0,1,219348,1,1,1,111
1000104,159.0,2,1070546,2,2,4,224
1000105,5.0,5,3697893,4,4,4,444
1000106,69.0,2,1112120,3,2,4,324
1000107,192.0,3,1027622,1,4,3,143
1000108,180.0,3,644940,1,4,2,142
1000109,96.0,2,1119154,3,2,4,324


In [27]:
rfmSeg['RFMClass'] = rfmSeg.R_Quartile.map(str) \
                            + rfmSeg.F_Quartile.map(str) \
                            + rfmSeg.M_Quartile.map(str)
rfmSeg.head(10)

,recency,frequency,monetary_value,R_Quartile,F_Quartile,M_Quartile,RFMClass
CardID,,,,,,,
1000101,98.0,1,157999,3,1,1,311
1000102,206.0,3,1750216,1,4,4,144
1000103,338.0,1,219348,1,1,1,111
1000104,159.0,2,1070546,2,2,4,224
1000105,5.0,5,3697893,4,4,4,444
1000106,69.0,2,1112120,3,2,4,324
1000107,192.0,3,1027622,1,4,3,143
1000108,180.0,3,644940,1,4,2,142
1000109,96.0,2,1119154,3,2,4,324


In [28]:
print("Pelanggan Tetap: ",len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Langganan: ',len(rfmSeg[rfmSeg['F_Quartile']==4]))
print("Pembeli banyak: ",len(rfmSeg[rfmSeg['M_Quartile']==4]))
print("Beresiko: ",len(rfmSeg[rfmSeg['R_Quartile']==1]))
print('Lost: ', len(rfmSeg[rfmSeg['RFMClass']=='111']))
print('Pelanggan berpotensial: ', len(rfmSeg[rfmSeg['RFMClass']=='333']) + len(rfmSeg[rfmSeg['RFMClass']=='444']))
print('Butuh Perhatian: ', len(rfmSeg[rfmSeg['RFMClass']=='222']) + len(rfmSeg[rfmSeg['RFMClass']=='111']))

Pelanggan Tetap:  9
Langganan:  36
Pembeli banyak:  39
Beresiko:  39
Lost:  8
Pelanggan berpotensial:  9
Butuh Perhatian:  8
